In [ ]:
import os
import shutil

import rasterio as rio

import seabeepy as sb
from seabeepy.config import SETTINGS

In [ ]:
# Login to MinIO
minio_client = sb.storage.minio_login(
    user=SETTINGS.MINIO_ACCESS_ID, password=SETTINGS.MINIO_SECRET_KEY
)

# Update raster band information

Some old raster mosaics (e.g. from Pix4D) have non-standard band order and no metadata. As a result, they display oddly when published to GeoNode. This script updates the band info so they can be processed correctly.

In [ ]:
temp_fold = r"/home/notebook/temp"
ds_list = [
    r"/home/notebook/shared-seabee-ns9879k/niva-tidy/2022/spectrofly_202208181238_kelpmap_vega-s_msi_120/orthophoto/pix4d_orthophoto.original.tif",
    r"/home/notebook/shared-seabee-ns9879k/niva-tidy/2022/spectrofly_202208190819_kelpmap_vega-n_msi_120/orthophoto/pix4d_orthophoto.original.tif",
    r"/home/notebook/shared-seabee-ns9879k/niva-tidy/2022/niva_202208181012_kelpmap_vega-s_msi_60/orthophoto/pix4d_orthophoto.original.tif",
    # r"/home/notebook/shared-seabee-ns9879k/niva-tidy/2022/spectrofly_202208181139_kelpmap_vega-s_msi_120/orthophoto/pix4d_orthophoto.original.tif",
    # r"/home/notebook/shared-seabee-ns9879k/niva-tidy/2022/spectrofly_202208180945_kelpmap_vega-s_msi_120/orthophoto/pix4d_orthophoto.original.tif",
    # r"/home/notebook/shared-seabee-ns9879k/niva-tidy/2022/niva_202208190944_kelpmap_vega-n_msi_120/orthophoto/pix4d_orthophoto.original.tif",
    # r'/home/notebook/shared-seabee-ns9879k/niva-tidy/2022/spectrofly_202208181357_kelpmap_vega-s_msi_198/orthophoto/pix4d_orthophoto.original.tif',
]

# Properties to set for all mosaics in 'ds_list'
band_order = ["blue", "green", "red", "rededge", "nir"]
# band_order = [None, None, None, None, None]

In [ ]:
for ds_path in ds_list:
    # Make a copy of the original file for editing
    temp_tif = os.path.join(temp_fold, "temp.tif")
    shutil.copyfile(ds_path, temp_tif)

    # Update band info
    with rio.open(temp_tif, "r+") as ds:
        nbands = ds.count
        for bidx in range(nbands):
            ds.set_band_description(bidx + 1, band_order[bidx])

    # Copy back
    sb.storage.copy_file(temp_tif, ds_path, minio_client, overwrite=True)
    os.remove(temp_tif)